Let's see if we can replicate some results of 'extracting paragraphs'

In [1]:
import torch 
import math
from nnsight import CONFIG
import nnsight
import numpy as np 
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\durrc\OneDrive\Desktop\mech_interp_work\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from activation_transplanting import *

In [3]:
# read the api_key
with open("../api_key_for_ndif.txt", "r") as file:
   ndif_key=file.read()

CONFIG.set_default_api_key(os.environ.get('NDIF_KEY'))

In [4]:
import os

# read the api_key
with open("../huggingface_token.txt", "r") as file:
    hf_key=file.read()

# Llama 3.1 8b is a gated model, so you need to apply for access on HuggingFace and include your token.
os.environ['HF_TOKEN'] = os.environ.get('HF_TOKEN')

## The Experiment 

Confirm the basic findings of `Extracting Paragraphs` 

Finding 1:
> Attention is focused around the `\n\n` tokens, after an instruction

Finding 2:
> By replacing the activations at newlines with another set of activations, we can make the model reproduce the original prompt 

In [5]:
from nnsight import LanguageModel
# We'll never actually load the parameters locally, so no need to specify a device_map.
llama = LanguageModel("meta-llama/Meta-Llama-3.1-8B")
# All we need to specify using NDIF vs executing locally is remote=True.

TODO: get temperature in the config 

In [6]:
num_examples = 1

with open("context.txt", 'r') as file:
    context = file.read()

context = "\n--\n".join(context.split("\n--\n")[:num_examples])+"\n--\n"

In [7]:
context

'What are the key elements of photosynthesis?\n\nPhotosynthesis is the process where plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It occurs in chloroplasts containing chlorophyll, which captures light energy. The process includes light-dependent reactions producing ATP and NADPH, and the Calvin cycle that uses these to fix carbon dioxide into sugar.\n--\n'

In [8]:
s1 = "What was the cause of the Bubonic Plague?\n\n"
s2 = "What were the causes of the Civil War?\n\n"


In [9]:
print(context+s1)

What are the key elements of photosynthesis?

Photosynthesis is the process where plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It occurs in chloroplasts containing chlorophyll, which captures light energy. The process includes light-dependent reactions producing ATP and NADPH, and the Calvin cycle that uses these to fix carbon dioxide into sugar.
--
What was the cause of the Bubonic Plague?




In [10]:
tk = LLamaExamineToolkit(llama_model=llama, remote=True, num_prev=0)

In [ ]:
outputs = tk.transplant_newline_activities(source_strings=[context+s1], target_strings=[context+s2], num_new_tokens=50, index=-1)

extracting newline activations


2025-03-03 20:00:18,738 40b8c546-bef1-480e-b309-540eef861675 - RECEIVED: Your job has been received and is waiting approval.
2025-03-03 20:00:19,326 40b8c546-bef1-480e-b309-540eef861675 - APPROVED: Your job was approved and is waiting to be run.


In [2]:
for out in outputs:
    print(out)
    print("--------------")

NameError: name 'outputs' is not defined